In [5]:
from dotenv import load_dotenv
import os

from typing import TypedDict, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

from gen_test_cases import generate_test_cases_from_requirements
from gen_requirements import generate_requirement_from_doc 

load_dotenv()
llm_api_key = os.getenv("GOOGLE_API_KEY")

In [6]:

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-04-17",
    temperature=0.3,
    max_tokens = None,
    api_key = llm_api_key,
)

In [7]:
class Project():
    name: str
    description: str
    requirements: list
    testCases: list

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    projects: list[Project]

In [8]:
@tool
def generate_requirements_from_document_pdf_tool(
    x: Annotated[str, "base64 encoded string representation of a pdf file"],
) -> tuple[list,str]:
    """Creating a brief summary of the document and a list of requirements from the project document in base64 encoded string representation of a pdf file """

    return generate_requirement_from_doc(x)


NameError: name 'tool' is not defined

In [ ]:
@tool
def generate_testCases_fromRequirements_tool(
    requirements: Annotated[list, "list of requirements from a project document format in Python list structure"],
    context: Annotated[str, "Project context of the requirements to better understand them"]
) -> list:
    """Creating a list of test cases from a list of requirements provided with test case"""
    
    return generate_test_cases_from_requirements(requirements, context)

In [ ]:

tools = [generate_testCases_fromRequirements_tool,generate_requirements_from_document_pdf_tool]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing QA in SDLC.")

def call_model(state: AgentState):
    return {"messages": [llm_with_tools.invoke(state{"messages"})]}

In [ ]:
builder = StateGraph(AgentState)
builder.add_node("call_model", call_model)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START,"call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools","call_model")
builder.add_edge("call_model", END)

graph = builder.compile()

In [ ]:
graph.invoke({"messages": HumanMessage(content="Hello")})